# Estimates with tools producing QIR

Azure Quantum Resource Estimation is built upon [QIR](https://www.qir-alliance.org/), the forward-looking, fully interoperable specification for quantum programs.  In this notebook, we are showing how to use the `azure.quantum` Python package to directly submit QIR to the Resource Estimation target.  We are using [PyQIR](https://github.com/qir-alliance/pyqir) to generate QIR, however the example works with any other source of QIR as well. PyQIR can help you as a library to generate QIR from other quantum programming languages, and thereby enabling their execution on Azure Quantum Resource Estimator.

## Getting started

We import several Python classes and functions from `azure.quantum` and `pyqir`.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator

# Support code to transition from pyqir-generator to pyqir package
try:
    from pyqir.generator import BasicQisBuilder, SimpleModule
except:
    from pyqir import BasicQisBuilder, SimpleModule

We connect to the Azure Quantum workspace.

In [ ]:
workspace = Workspace(
    resource_id = "",
    location = ""
)

We create an instance of the Resource Estimator in that workspace.  Make sure that you have the _Microsoft Quantum Computing_ provider added to the workspace.

In [ ]:
estimator = MicrosoftEstimator(workspace)

## Running a sample quantum program

Let's now create some QIR bitcode using PyQIR generator.  Here, we build a controlled S gate using 3 T gates and 2 CNOT gates.

In [ ]:
module = SimpleModule("Controlled S", num_qubits=2, num_results=0)
qis = BasicQisBuilder(module.builder)

[a, b] = module.qubits[0:2]
qis.t(a)
qis.t(b)
qis.cx(a, b)
qis.t_adj(b)
qis.cx(a, b)

Before we submit the QIR to the resource estimator, let's take a look at the QIR
output for this module.  We can use the `ir` function from PyQIR for that
purpose, that generates human-readable instructions.

In [ ]:
print(module.ir())

The QIR we submit to the function defined above must be passed in bitcode
format.  We obtain this format by calling `bitcode` instead of `ir` on the
module.  We can also pass resource estimation specific arguments, e.g., setting
the error rate to 0.5%.

In [ ]:
params = estimator.make_params()
params.error_budget = 0.005
job = estimator.submit(module.bitcode(), input_params=params)
result = job.get_results()

Finally, we print the resource estimation table.

In [ ]:
result